In [4]:
# autoRMZ test/development notebook (v1).

In [5]:
# List all files to ensure all dependencies exist.
!ls

LICENSE    dataset    processor  test.png   v1.ipynb
README.md  pipeline   setup.py   testing.py


In [6]:
# Import dependencies
import cv2
import json
from pathlib import Path

In [7]:
# Create data paths
preprocessed_path = Path('./dataset/preprocessed')
processed_path = Path('./dataset/processed')

In [8]:
# Get files from data paths
preprocessed_images = [e for e in preprocessed_path.iterdir() if e.is_file()]
processed_json = [e for e in processed_path.iterdir() if e.is_file()]

In [12]:
# Crop preprocessed data with processed data to obtain dataset to be used for training.
if not Path('./dataset/roi').is_dir():
    Path('./dataset/roi').mkdir()

for json_file in processed_json:
    data_point_name = json_file.name[:-5]
    image_file = Path('./dataset/preprocessed/{}.png'.format(data_point_name))
    crop_areas = []
    with json_file.open() as f:
        crop_areas = json.load(f)['areas']
    
    image_path = str(image_file.absolute())
    im = cv2.imread(image_path)

    im_path = Path('./dataset/roi/{}'.format(data_point_name))

    if not im_path.is_dir():
        im_path.mkdir()
    
    for j in range(0, len(crop_areas)):
        area = crop_areas[j]
        x = min(area['p1'][0], area['p2'][0])
        y = min(area['p1'][1], area['p2'][1])
        width = abs(area['p1'][0] - area['p2'][0])
        height = abs(area['p1'][1] - area['p2'][1])
        # print(x, y, x+width, y+height)
        roi = im[y:y+height, x:x+width]
        roi_path = Path('./dataset/roi/{}/{}.png'.format(data_point_name, j)).absolute()
        cv2.imwrite(str(roi_path), roi)

    # break